In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd

In [16]:
driver = webdriver.Chrome()

In [17]:
# go to the website
url = "https://www.transfermarkt.com/saudi-pro-league/gesamtspielplan/wettbewerb/SA1/saison_id/2024"
driver.get(url)

# wait for the cookies box
time.sleep(5)

In [ ]:
# refresh and scroll the page manually 
driver.refresh()

In [21]:
html_source = driver.page_source

In [22]:
driver.quit()

In [48]:
soup = BeautifulSoup(html_source, "html.parser")
matchday_tables = soup.find_all("div", class_="large-6 columns") + soup.find_all("div", class_="large-6 columns end")

In [107]:
filtered_tables = [t.find_all("div", class_="box") for t in matchday_tables]

In [109]:
# df columns
dates, times, home_teams, results, away_teams = [], [], [], [], []

In [115]:
for table in filtered_tables:
    rows = table[0].find_all("tr")
    for row in rows:
        # Extract date (if present)
        date_tag = row.find("td", class_="hide-for-small")
        date = date_tag.find("a").text.strip() if date_tag and date_tag.find("a") else "N/A"

        # Extract time (if present)
        time_tag = row.find("td", class_="zentriert hide-for-small")
        time = time_tag.text.strip() if time_tag else "N/A"

        # Extract home team (if present)
        home_team_tag = row.find("td", class_="text-right no-border-rechts hauptlink")
        home_team = home_team_tag.text.strip() if home_team_tag else "N/A"

        # Extract result (if present)
        result_tag = row.find("td", class_="zentriert hauptlink")
        result = result_tag.find("a").text.strip() if result_tag and result_tag.find("a") else "N/A"

        # Extract away team (if present)
        away_team_tag = row.find("td", class_="no-border-links hauptlink")
        away_team = away_team_tag.text.strip() if away_team_tag else "N/A"

        # Append extracted data
        dates.append(date)
        times.append(time)
        home_teams.append(home_team)
        results.append(result)
        away_teams.append(away_team)

In [116]:
# Create a DataFrame
df = pd.DataFrame({
    "Date": dates,
    "Time": times,
    "Home Team": home_teams,
    "Result": results,
    "Away Team": away_teams
})

In [131]:
df.head()

,Date,Time,Home Team,Result,Away Team
0,N/A,N/A,N/A,N/A,N/A
1,N/A,N/A,N/A,N/A,N/A
2,8/22/24,6:10 PM,Al-Taawoun,1:0,Al-Fayha
3,N/A,N/A,N/A,N/A,N/A
4,N/A,8:00 PM,Al-Wehda,3:3,Al-Riyadh


In [ ]:
df.to_csv("uncleaned_saudi_pro_league_matches.csv", index=False)